In [1]:
import madmom as mm
import numpy as np
from madmom.features.beats import RNNBeatProcessor
import madmom

In [2]:
act = RNNBeatProcessor()('Not A Romance.wav')

/anaconda/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [3]:
proc = madmom.features.beats.BeatTrackingProcessor(fps=100)

In [4]:
stuff = proc(act)

In [5]:
intervals = []
for i in range(len(stuff)-1):
    intervals.append(stuff[i+1] - stuff[i])

from collections import Counter
data = Counter(intervals)
qqq = 0
iii = 0
for x in data.most_common():
    qqq += x[0]*x[1]#*x[1]
    iii += x[1]#*x[1]
qqq /= iii
print(60/qqq)

163.990175704


In [6]:
proc1 = RNNBeatProcessor()
probs = proc1('Not A Romance.wav')  

/anaconda/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [7]:
print(max(probs))
bpm = float(round(60/qqq))
print(bpm)

0.707564
164.0


In [ ]:
for i in range(len(probs)):
    if probs[i] >= max(probs): 
        offset = float(i)/float(100)
print(offset)

151.15


In [ ]:
h = offset
while h >= 0:
    if h - 60.0/bpm-0.003 >= 0:
        h = h - 60.0/bpm-0.003
    
print(h)

In [ ]:
print(stuff[0:10])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import wave
import sys


spf = wave.open('Not a Romance.wav','r')

#Extract Raw Audio from Wav File
signal = spf.readframes(-1)
signal = np.fromstring(signal, 'Int16')


#If Stereo
#if spf.getnchannels() == 2:
 #   sys.exit(0)

plt.figure(1)
plt.title('Signal Wave...')
plt.plot(signal)
plt.show()

In [ ]:
# We'll need numpy for some mathematical operations
import numpy as np

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

# and IPython.display for audio output
import IPython.display

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

In [ ]:
audio_path = 'Not A Romance.wav'

y, sr = librosa.load(audio_path)

In [ ]:
y_harmonic, y_percussive = librosa.effects.hpss(y)

In [ ]:
# What do the spectrograms look like?
# Let's make and display a mel-scaled power (energy-squared) spectrogram
S_harmonic   = librosa.feature.melspectrogram(y_harmonic, sr=sr)
S_percussive = librosa.feature.melspectrogram(y_percussive, sr=sr)

# Convert to log scale (dB). We'll use the peak power as reference.
log_Sh = librosa.power_to_db(S_harmonic, ref=np.max)
log_Sp = librosa.power_to_db(S_percussive, ref=np.max)

# Make a new figure
plt.figure(figsize=(12,6))

plt.subplot(2,1,1)
# Display the spectrogram on a mel scale
librosa.display.specshow(log_Sh, sr=sr, y_axis='mel')

# Put a descriptive title on the plot
plt.title('mel power spectrogram (Harmonic)')

# draw a color bar
plt.colorbar(format='%+02.0f dB')

plt.subplot(2,1,2)
librosa.display.specshow(log_Sp, sr=sr, x_axis='time', y_axis='mel')

# Put a descriptive title on the plot
plt.title('mel power spectrogram (Percussive)')

# draw a color bar
plt.colorbar(format='%+02.0f dB')

# Make the figure layout compact
plt.tight_layout()

In [ ]:
from scipy.signal import savgol_filter

ack = []
pe = log_Sp.transpose()
print(pe.shape)
for i in range(pe.shape[0]):
    ack.append(pe[i][100:])
#print(ack)
sums = []
print(len(ack))
for i in range(len(ack)):
    sums.append(ack[i].sum())
m = min(sums)

for i in range(len(sums)):
    sums[i] -= m
    sums[i] = sums[i]**2
#for i in range(len(sums)):
 #   sums[i] -= sums.sum()/len(sums)

ack2 = savgol_filter(sums, window_length=51, polyorder=2) 
#ack2 = sums

#for i in range(len(sums)):
 #   sums[i] -= m*2
print(max(ack2), min(ack2), max(ack2)/2, (max(ack2)+min(ack2))/2)
import matplotlib.pyplot as plt
plt.figure(figsize=(30,15))
plt.plot(ack2)

plt.show()

In [ ]:
beats_count = frames_to_time(len(ack2))

def time_to_beat(bpm, first_beat, timestamp):
    apprx_beat = (timestamp - first_beat)*(bpm/60.0)*4
    #print(apprx_beat, timestamp)
    return apprx_beat
    

In [ ]:
import gf10
import pickle

with open('markov_steps.pkl', 'rb') as fid:
    model = pickle.load(fid)

def make_stream(measure_count, beat):
    line = gf10.generator3000(total = measure_count, tact = beat, text_model = model)
    final_line=gf10.unchainus(line)
    stream = final_line.split('\n')[:-1]
    return stream


In [ ]:
from librosa.core import frames_to_time
#from statistics import median_low
idea = []

#mean_mel = median_low(ack2)
mean_mel = (max(ack2) + min(ack2))/2
print(mean_mel)
stream_flag = 0
stream = []
ass = []
for lvl in range(len(ack2)):
    if ack2[lvl] > mean_mel and stream_flag == 0:
        stream_flag = 1
        stream.append(lvl)
        ass.append
    elif ack2[lvl] < mean_mel and stream_flag == 1:
        stream_flag = 0
        stream.append(lvl)
        stream = frames_to_time(stream[:])
        #print(stream)
        idea.append(stream)
        stream = []
        

for k in idea:
    k[0] = round(time_to_beat(164, 0.20287804878180116, k[0]))
    k[1] = round(time_to_beat(164, 0.20287804878180116, k[1]))
    #print(k)
i = 1
while i < len(idea):
    if (idea[i][0]-1 == idea[i-1][1]) or (idea[i][0] == idea[i-1][1]):
        idea[i-1][1] = idea[i][1]
        idea.pop(i)
    else:
        i += 1

for k in idea:
    print(k)
        

In [ ]:
bpm = 164.0
track_len = (frames_to_time(len(ack2))-0.203)/60.0*bpm
print(frames_to_time(len(ack2))-0.203)
print(frames_to_time(len(ack2)), track_len)
b = []
stream_flag = 0
stream = []

for i in range(int(track_len)*4):
    
    for j in range(len(idea)):
        if i == int(idea[j][0]):
            stream_flag = 1
            stream = make_stream(idea[j][1]-idea[j][0], 16)
    k = 0
    if stream_flag == 0:
        b.append('0000')
    else:
        b.append(stream[k])
        k += 1
    
    for j in range(len(idea)):
        if i == int(idea[j][1]):
            stream_flag = 0
    print(i, stream_flag)
    if i%16 == 0:
        b.append(',')

In [ ]:
f = open('zzz.txt', 'w')
for s in b:
    f.write(s+'\n')
    print(s)
    
f.close()

In [ ]:
make_stream(2, 4)
